In [2]:
from midi_tokenizer import MIDITokenizer
import MIDI 
# Note: The original MIDI.py has poor performance because list.pop(0) has poor performance on some pythons or machines. I changed it to list = list[1:]
import glob
import re
from tqdm.notebook import tqdm
from pathlib import Path
import shutil
import random

In [3]:
tokenizer = MIDITokenizer()

def process_midi_file(midi_file):
    try:
        with open(midi_file, 'rb') as f:
            datas = f.read()
    except Exception:
        print(midi_file)
        return
    res = []
    if len(datas) > 384000:  # file too large that can not load by `MIDI.midi2score`
        res.append("large")
    elif len(datas) < 3000:
        res.append("small")
    if not res:
        try:
            mid = MIDI.midi2score(datas)
            mid = tokenizer.tokenize(mid)
            quality, res = tokenizer.check_quality(mid)
        except Exception:
            quality = False
            res = ["unk"] # broken midi file or Exception in the code
    else:
        quality = False
        
    if quality:
        path = midi_file.replace(dataset_dir, f"{processed_dir}/")
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        shutil.move(midi_file, path)
    else:
        res = "_".join(res)
        path = midi_file.replace(dataset_dir, f"{rm_dir}/{res}/")
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        shutil.move(midi_file, path)


def process_all(midi_files):
    for midi_file in tqdm(midi_files):
        process_midi_file(midi_file)

In [4]:
dataset_dir = r"D:\myprojects\dataset\midi_datasets"
processed_dir = r"D:\myprojects\dataset\midi_datasets_processed" # All processed midi will be moved to here. The folder will be created automatically.
rm_dir = r"D:\myprojects\dataset\midi_datasets_rm" # All bad midi will be moved here. The folder will be created automatically.

In [5]:
midi_files = glob.glob(f"{dataset_dir}/**/*", recursive=True)
midi_files = [file for file in midi_files if re.search(r'\.midi?$', file, re.IGNORECASE)]
midi_files = sorted(midi_files)

In [6]:
# check if dataset_dir is correct
print(len(midi_files))
print(random.choice(midi_files))

203144
D:\myprojects\dataset\midi_datasets\los_midi\MIDIs\1\127e2700fa41458e5cbfde61066eeb33.mid


In [7]:
# start processing. You can increase the thread num if your machine is good
# If you cancel the run, you will need to reload midi_files before you can run it again
process_all(midi_files)

  0%|          | 0/203144 [00:00<?, ?it/s]

KeyboardInterrupt: 